In [1]:
import pandas as pd
from datetime import datetime
import random

In [4]:
df1 = pd.read_csv('Data/GOES_16_merged_data.csv')
df2 = pd.read_csv('Data/GOES_17_merged_data.csv')
df3 = pd.read_csv('Data/GOES_18_merged_data.csv')

/var/folders/r7/6cts6h556zzgbx26m_g0p4gw0000gn/T/ipykernel_6517/2651289391.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Data/GOES_17_merged_data.csv')


In [5]:
df1.head(1000)

,time,xrsa_flux,xrsb_flux,status,flare_class
0,2017-02-07 00:00:00,2.949822e-08,7.072423e-08,NaN,NaN
1,2017-02-07 00:01:00,2.716477e-08,6.869706e-08,NaN,NaN
2,2017-02-07 00:02:00,2.996076e-08,6.933664e-08,NaN,NaN
3,2017-02-07 00:03:00,3.124979e-08,6.796333e-08,NaN,NaN
4,2017-02-07 00:04:00,3.095799e-08,6.928261e-08,NaN,NaN
...,...,...,...,...,...
995,2017-02-07 16:35:00,4.418819e-09,4.118933e-08,NaN,NaN
996,2017-02-07 16:36:00,3.437882e-09,3.999427e-08,NaN,NaN
997,2017-02-07 16:37:00,6.502486e-09,4.040907e-08,NaN,NaN
998,2017-02-07 16:38:00,5.297591e-09,3.889536e-08,NaN,NaN


In [6]:
merged_df = pd.merge(df1, df2, on='time', how='outer')
merged_df = pd.merge(merged_df, df3, on='time', how='outer')
print(len(merged_df))


3548372


In [7]:
merged_df = merged_df[["time","xrsa_flux_x","xrsb_flux_x","status_x","flare_class_x"]]
merged_df = merged_df.rename(columns={"xrsa_flux_x": "xrsa_flux", "xrsb_flux_x": "xrsb_flux", 'status_x':'status', 'flare_class_x': 'flare_class'})

mask = merged_df['xrsa_flux'] != merged_df['xrsa_flux'].isna

merged_df = merged_df[mask]

merged_df = merged_df.drop_duplicates(subset='time', keep='first')
print(len(merged_df))

3548160


In [8]:
print(merged_df["time"].is_unique)

True


In [9]:
duplicates = merged_df[merged_df.duplicated('time', keep=False)]

# Display duplicate rows
print("Duplicate Time Values:")
print(duplicates)
print(len(duplicates))
print(len(merged_df))
has_duplicates = merged_df['time'].duplicated().any()
print(f"Are there duplicate time values? {has_duplicates}")

Duplicate Time Values:
Empty DataFrame
Columns: [time, xrsa_flux, xrsb_flux, status, flare_class]
Index: []
0
3548160
Are there duplicate time values? False


In [10]:
first_value = merged_df.head(1)
first_value_time = str(first_value['time'].item())
first_value_time = datetime.strptime(first_value_time, '%Y-%m-%d %H:%M:%S')
print(first_value_time)

2017-02-07 00:00:00


In [11]:
last_value = merged_df.tail(1)
last_value_time = str(last_value['time'].item())
last_value_time = datetime.strptime(last_value_time, '%Y-%m-%d %H:%M:%S')
print(last_value_time)

2023-11-06 23:59:00


In [12]:
time_difference = last_value_time - first_value_time
amount_of_minutes = int(time_difference.total_seconds() / 60)
print(amount_of_minutes)

3548159


In [13]:
print(len(merged_df))
print()

3548160


In [14]:
merged_df.to_csv("full_set.csv", index=False)

# Transforming the Dataset from long format to wide  Format 

In [27]:
merged_df = pd.read_csv('full_set.csv')

In [28]:
merged_df = merged_df[["time","xrsa_flux","status"]]

In [29]:
in_event = False
event_label = 0
labels = []

for status in merged_df['status']:
    if status == 'EVENT_PEAK':
        event_label = 1
    else:
        event_label = 0
    labels.append(event_label)

cleaned = merged_df[["time","xrsa_flux"]]
cleaned['Label'] = labels

# removing all the 2017 data
cleaned['time'] = pd.to_datetime(cleaned['time'])

start_time = pd.to_datetime('2018-01-01 00:00:00')
cleaned = cleaned[(cleaned['time'] >= start_time)]

cleaned = cleaned.dropna(subset=['xrsa_flux'])
cleaned = cleaned.reset_index(drop=True)

sum(cleaned.Label)

# cleaned.head(10000)  

7573

In [30]:
sum(cleaned.Label)
peaks = []
non_peaks = []

window_size = 50 # will result in a window of size 100 

for i  in range(window_size, len(cleaned) - window_size):
    window = cleaned.iloc[i- window_size : i+ window_size + 1]
    if window.Label[i] == 1:
        time = cleaned.iloc[i]['time']  # Capture the time of the peak
        row = [time] + list(window['xrsa_flux'].reset_index(drop=True))
        row.append(1)  # Label for peak
        peaks.append(row)
    if window.Label[i] == 0: 
        time = cleaned.iloc[i]['time']  # Capture the time of the peak
        row = [time] + list(window['xrsa_flux'].reset_index(drop=True))
        row.append(0)
        non_peaks.append(row)

In [31]:
print(len(peaks))
print(len(non_peaks))

mix = peaks + non_peaks[: (len(peaks) * 3)]

7573
2977318


In [32]:
random.shuffle(mix)
new_wide_series_list = mix

In [33]:
print(len(new_wide_series_list))
print(type(new_wide_series_list[0]))

new_transposed_df = pd.DataFrame(new_wide_series_list)

30292
<class 'list'>


In [35]:
new_transposed_df.columns = ['time'] + [str(i) for i in range((window_size * 2) + 1)] + ['Label'] 

In [36]:
new_transposed_df.to_csv("wide_100_timein.csv", index=False)

## converting code to a function that takes input window size

In [2]:
def window_creater(file_path='full_set.csv' ,window_size=50, non_peak_proportion=3):
    """
    :param file_path: default is 'full_set.csv'. It's the full dataset of merged readings from 3 satellites in the long format. 
    :param window_size: default is 50. Integer How many Readings should sandwich the peak value (center)
    :param non_peak_proportion: default is 3. How many non-peaks should we have for every peak  
    :return: A dataframe that is in the wide format with columns "time", [n amount of readings], "label"
    """
    source_data = pd.read_csv(file_path)
    source_data = source_data[["time","xrsa_flux","status"]]
    labels = []
    
    for status in source_data['status']:
        if status == 'EVENT_PEAK':
            event_label = 1
        else:
            event_label = 0
        labels.append(event_label)
    
    cleaned = source_data[["time","xrsa_flux"]]
    cleaned['Label'] = labels
    
    # removing all the 2017 data
    cleaned['time'] = pd.to_datetime(cleaned['time'])
    
    start_time = pd.to_datetime('2018-01-01 00:00:00')
    cleaned = cleaned[(cleaned['time'] >= start_time)]
    
    cleaned = cleaned.dropna(subset=['xrsa_flux']).reset_index(drop=True)
    
    peaks = []
    non_peaks = []
    
    for i  in range(window_size, len(cleaned) - window_size):
        window = cleaned.iloc[i- window_size : i+ window_size + 1]
        if window.Label[i] == 1:
            time = cleaned.iloc[i]['time']  # Capture the time of the peak
            row = [time] + list(window['xrsa_flux'].reset_index(drop=True))
            row.append(1)  # Label for peak
            peaks.append(row)
        if window.Label[i] == 0: 
            time = cleaned.iloc[i]['time']  # Capture the time of the peak
            row = [time] + list(window['xrsa_flux'].reset_index(drop=True))
            row.append(0)
            non_peaks.append(row)
            
    mixed_peaks = peaks + non_peaks[: (len(peaks) * non_peak_proportion)]
    random.shuffle(mixed_peaks)

    final_transposed_df = pd.DataFrame(mixed_peaks)
    final_transposed_df.columns = ['time'] + [str(i) for i in range((window_size * 2) + 1)] + ['Label']
    
    return final_transposed_df
    

In [3]:
# Testing to hopefully get the same results from earlier 
output_df = window_creater()
counts = output_df['Label'].value_counts()

print(counts)

Label
0    22719
1     7573
Name: count, dtype: int64
